# Example: parsing FHIR feeds with DuckDB and Python

In [1]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Allow named parameters (python variables) in SQL cells
%config SqlMagic.named_parameters=True

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


/Users/me/jaanli/synthetic-healthcare-data/.venv/lib/python3.12/site-packages/sql/traits.py:20: FutureWarning: named_parameters: boolean values are now deprecated. Value True will be treated as "enabled". 
Please use a valid option: "warn", "enabled", or "disabled". 
For more information, see the docs: https://jupysql.ploomber.io/en/latest/api/configuration.html#named-parameters
  warnings.warn(


In [3]:
%%sql 

SELECT * FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json')

RuntimeError: (duckdb.duckdb.BinderException) Binder Error: Function "read_json_auto" is a table function but it was used as a scalar function. This function has to be called in a FROM clause (similar to a table).
LINE 1: SELECT * FROM unnest(read_json_auto('data/Gudrun69_Shaunna80...
                             ^
[SQL: SELECT * FROM unnest(read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json'))]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Claude prompt

````
take this simple duckdb command: 

%%sql 

SELECT * FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json')

revise it using the above blog post attached, and the raw data, also attached a snippet of this in json format, and potentially the unnest function (documentation attached for duckdb), alongside the JSON duckdb documentation attached.

proceed step-by-step, as you are a principal software engineer at google focused on health care interoperability. 
````

Context: 

https://duckdb.org/2023/03/03/json.html
https://duckdb.org/docs/extensions/json.html
https://duckdb.org/docs/data/json/overview.html

In [19]:
!head -n 1000 data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json

{
  "resourceType": "Bundle",
  "type": "transaction",
  "entry": [ {
    "fullUrl": "urn:uuid:d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e",
    "resource": {
      "resourceType": "Patient",
      "id": "d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e",
      "meta": {
        "profile": [ "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient" ]
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</a>.Version identifier: master-branch-latest\n .   Person seed: -3660445453666907032  Population seed: 1712341668451</div>"
      },
      "extension": [ {
        "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race",
        "extension": [ {
          "url": "ombCategory",
          "valueCoding": {
            "system": "urn:oid:2.16.840.1.113883.6.238",
            "code": "2028-9",
            "display": "Asian"
          }
        }, {
    

In [14]:
%%sql 

SELECT *
FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json', columns={'entry': 'JSON[]'}) AS entries;

,entry
0,"[{""fullUrl"":""urn:uuid:d5e33bd1-960e-bcf4-e5f9-..."


In [16]:
%%sql 
SELECT unnested_entry
FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json', columns={'entry': 'JSON[]'}) AS entries,
     unnest(entries.entry) AS unnested_entry;

,unnested_entry
0,"{'entry': '{""fullUrl"":""urn:uuid:d5e33bd1-960e-..."
1,"{'entry': '{""fullUrl"":""urn:uuid:02fc23fb-e750-..."
2,"{'entry': '{""fullUrl"":""urn:uuid:721d6cf7-fb60-..."
3,"{'entry': '{""fullUrl"":""urn:uuid:f9313a3a-b83c-..."
4,"{'entry': '{""fullUrl"":""urn:uuid:84a58c28-da98-..."
...,...
665,"{'entry': '{""fullUrl"":""urn:uuid:30d10eb8-8d97-..."
666,"{'entry': '{""fullUrl"":""urn:uuid:2bd36759-c5eb-..."
667,"{'entry': '{""fullUrl"":""urn:uuid:4a82b823-20fc-..."
668,"{'entry': '{""fullUrl"":""urn:uuid:55643c5e-bde8-..."


In [23]:
%%sql 
SELECT json_type(json(unnested_entry.entry)) AS entry_type,
       json_type(json_extract(json(unnested_entry.entry), '$.resource')) AS resource_type
FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json', columns={'entry': 'JSON[]'}) AS entries,
     unnest(entries.entry) AS unnested_entry
LIMIT 1;

,entry_type,resource_type
0,OBJECT,OBJECT


In [24]:
%%sql 
SELECT json_extract(json(unnested_entry.entry), '$.resource') AS resource
FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json', columns={'entry': 'JSON[]'}) AS entries,
     unnest(entries.entry) AS unnested_entry
WHERE json_extract_string(json(unnested_entry.entry), '$.resource.resourceType') = 'Patient';

,resource
0,"{""resourceType"":""Patient"",""id"":""d5e33bd1-960e-..."


In [25]:
%%sql 
SELECT
    json_extract(resource, '$.name[0].given[0]') AS first_name,
    json_extract(resource, '$.name[0].family') AS last_name,
    json_extract_string(resource, '$.gender') AS gender,
    json_extract_string(resource, '$.birthDate') AS birth_date
FROM (
    SELECT json_extract(json(unnested_entry.entry), '$.resource') AS resource
    FROM read_json_auto('data/Gudrun69_Shaunna800_Goyette777_d5e33bd1-960e-bcf4-e5f9-9a4afc6d5a5e.json', columns={'entry': 'JSON[]'}) AS entries,
         unnest(entries.entry) AS unnested_entry
    WHERE json_extract_string(json(unnested_entry.entry), '$.resource.resourceType') = 'Patient'
) AS patient_resource;

,first_name,last_name,gender,birth_date
0,"""Gudrun69""","""Goyette777""",female,1979-11-07
